# BQML

Create a container for data, which expires after 7 days.

In [ ]:
!bq --location=US mk -d \
--default_table_expiration 604800 \
--description "Credit card fraud datasets and models" \
fraud

Load in the credit dataset

In [ ]:
!bq load \
--autodetect \
--source_format=CSV \
fraud.creditcard \
../dataset/creditcard.csv 

Load the testing dataset

In [ ]:
!bq load \
--autodetect \
--source_format=CSV \
fraud.creditcard_testing \
../dataset/testing.csv 

Load the training dataset

In [ ]:
!bq load \
--autodetect \
--source_format=CSV \
fraud.creditcard_training \
../dataset/training.csv 

## Train a model : logistic regression

 https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create

In [ ]:
%%bigquery
#standardSQL
CREATE OR REPLACE MODEL
  `fraud.fraud_model` OPTIONS(model_type='logistic_reg') AS
SELECT
  Class AS label,
  * -- V1-V28, Time and Amount are all inputs.
  EXCEPT (Class)
FROM
  `fraud.creditcard_training`

## Query model

https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create

In [ ]:
%%bigquery
#standardSQL
SELECT
  predicted_label,
  -- predicted_label_probs,
  label
FROM
  ML.PREDICT(MODEL `fraud.fraud_model`,
    (
    SELECT
      Class AS label,
      * -- V1-V28, Time and Amount are all inputs.
      EXCEPT (Class)
    FROM
      `fraud.creditcard_testing`) )